# Hydrogen Dispatch Example

This example demonstrates how to use the dispatch controller capability in H2Integrate.

In [ ]:
from pathlib import Path
from matplotlib import pyplot as plt
from h2integrate.core.h2integrate_model import H2IntegrateModel

In [ ]:
# Create a H2Integrate model
model = H2IntegrateModel(Path("./inputs/h2i_wind_to_h2_storage.yaml"))

# Run the model
model.run()
model.post_process()

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)

start_hour = 0
end_hour = 200
total_time_steps = model.technology_config["technologies"]["h2_storage"]["model_inputs"][
    "control_parameters"
]["time_steps"]
demand_profile = [
    model.technology_config["technologies"]["h2_storage"]["model_inputs"]["control_parameters"][
        "demand_profile"
    ]
    * 1e-3
] * total_time_steps

ax[0].plot(
    range(start_hour, end_hour),
    model.prob.get_val("h2_storage.hydrogen_soc", units="percent")[start_hour:end_hour],
    label="SOC",
)
ax[0].set_ylabel("SOC (%)")
ax[0].set_ylim([0, 110])

ax[1].plot(
    range(start_hour, end_hour),
    model.prob.get_val("h2_storage.hydrogen_in", units="t/h")[start_hour:end_hour],
    linestyle="-",
    label="H$_2$ In (kg)",
)
ax[1].plot(
    range(start_hour, end_hour),
    model.prob.get_val("h2_storage.hydrogen_curtailed", units="t/h")[start_hour:end_hour],
    linestyle=":",
    label="H$_2$ Curtailed (kg)",
)
ax[1].plot(
    range(start_hour, end_hour),
    model.prob.get_val("h2_storage.hydrogen_missed_load", units="t/h")[start_hour:end_hour],
    linestyle=":",
    label="H$_2$ Missed Load (kg)",
)
ax[1].plot(
    range(start_hour, end_hour),
    model.prob.get_val("h2_storage.hydrogen_out", units="t/h")[start_hour:end_hour],
    linestyle="-",
    label="H$_2$ Out (kg)",
)
ax[1].plot(
    range(start_hour, end_hour),
    demand_profile[start_hour:end_hour],
    linestyle="--",
    label="H$_2$ Demand (kg)",
)
ax[1].set_ylim([0, 30])
ax[1].set_ylabel("H$_2$ Hourly (t)")
ax[1].set_xlabel("Timestep (hr)")

plt.legend(ncol=2, frameon=False)